# Wykład 7 - podsumowanie

Krzysztof rusek


# Licencja

Copyright (c) 2020, Krzysztof Rusek
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

    * Redistributions of source code must retain the above copyright
      notice, this list of conditions and the following disclaimer.
    * Redistributions in binary form must reproduce the above copyright
      notice, this list of conditions and the following disclaimer in the
      documentation and/or other materials provided with the distribution.
    * Neither the name of the AGH nor the
      names of its contributors may be used to endorse or promote products
      derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL Krzysztof Rusek BE LIABLE FOR ANY
DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
(INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND
ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS
SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

# Praktyczne przykłady

- Analiza odpowiedzi testera dla Netflix
- Predykcja ruchu generowanego przez AGH

## Metody

- Zawansowane modelowanie probabilistyczne w tfp
- Szeregi czasowe w tfp


In [ ]:
#@title Import

import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


tfd = tfp.distributions
tfb=tfp.bijectors
Root = tfd.JointDistributionCoroutine.Root
sns.set()

# Netflix


Zakładamy:

1. Film ma jakąś jakość
1. Tester nie jest dokładny
1. Testerzy mogą byc surowi w ocenie

##### Model

 $$O_{ij}=\psi_j + \Delta_i + \upsilon_i X, \quad X \sim Norm(0,1)$$


In [ ]:
#@title Simulation parameters { display-mode:"form" }

#@markdown Number of PVSs
nPVS = 160 #@param
#@markdown Number of Subjects
nSub = 24 #@param

#@markdown A standard deviation for typical PVS
std = 0.75 #@param
#@markdown A parameter of the gamma distribution from which the variances are generated
alpha = 30.0 #@param
#@markdown Standard deviation for delta distribution
b = 0.3 #@param

In [ ]:
#@title Funkcje
def make_plot(psi,psi_hat,delta,delta_hat,upsilon,upsilon_hat,\
             psi_err=None,delta_err=None, upsilon_err=None):
    prop_cycle = plt.rcParams['axes.prop_cycle']
    colors = prop_cycle.by_key()['color']
    s=2.

    def diag(ax):
      ax.plot(ax.get_xlim(),ax.get_xlim(),color=colors[1])

    fig,ax = plt.subplots(nrows=2, ncols=2)
    fig.set_size_inches(10,10)

    ax[0,0].scatter(psi,psi_hat,zorder=3,s=s)
    #ax[0,0].plot([1,5],[1,5],color=colors[1],zorder=2)
    diag(ax[0,0])
    ax[0,0].set_xlabel(r'$\psi$')
    ax[0,0].set_ylabel(r'$\hat \psi$')

    if psi_err is not None:
        ax[0,0].errorbar(x=psi,y=psi_hat, fmt='none',yerr=psi_err, alpha=0.9,ecolor=colors[2],zorder=1)

    ax[0,1].scatter(delta,delta_hat,zorder=3,s=s)
    #ax[0,1].plot([-1,1.3],[-1,1.3],color=colors[1])
    diag(ax[0,1])
    ax[0,1].set_xlabel(r'$\delta$')
    ax[0,1].set_ylabel(r'$\hat \delta$')

    if delta_err is not None:
        ax[0,1].errorbar(x=delta,y=delta_hat, fmt='none',yerr=delta_err, alpha=0.9, ecolor=colors[2])

    ax[1,1].scatter(upsilon,upsilon_hat, zorder=3,s=s)
    #ax[1,1].plot([0.4,0.8],[0.4,0.8],color=colors[1])
    #ax[1,1].plot(ax[1,1].get_xlim(),ax[1,1].get_xlim(),color=colors[1])
    diag(ax[1,1])
    ax[1,1].set_xlabel(r'$\upsilon$')
    ax[1,1].set_ylabel(r'$\hat \upsilon$')

    if upsilon_err is not None:
        ax[1,1].errorbar(x=upsilon,y=upsilon_hat, fmt='none',yerr=upsilon_err, alpha=0.9,ecolor=colors[2],zorder=2)

In [ ]:
def model(quantized=True):
  delta = yield Root(
      tfd.Sample(tfd.Normal(loc=0.0,
      scale=0.6), (1, nSub) ) )

  psi = yield Root(tfd.Sample(tfd.Uniform(low = 1.0, high=5.0),
                   (nPVS,1)))

  rate = alpha/tf.math.sqrt(std**2 - b**2)

  upsilon = yield Root( tfd.Sample(tfd.Gamma(
      concentration=alpha,
      rate=rate,
  ),(1, nSub) )  )

  loc = psi+delta
  scale = upsilon

  c = tfd.TransformedDistribution(
      distribution=tfd.Normal(loc=loc, scale=scale),
      bijector=tfp.bijectors.Shift(shift=-0.5)

  )
  if quantized:
    q=tfd.QuantizedDistribution(
        distribution=c,
        low=1.0 ,
        high=5.0
    )
  else:
    q=c


  o = yield tfd.Independent(q, reinterpreted_batch_ndims=2)

In [ ]:
joint = tfd.JointDistributionCoroutine(model)

x = joint.sample()

def target_log_prob_fn(delta, psi, upsilon):
  """Unnormalized target density as a function of states."""
  return joint.log_prob((
      delta, psi, upsilon, x[3]))

In [ ]:
#@title Przykładowe dane
pd.DataFrame(joint.sample()[3].numpy().astype(np.int32))

In [ ]:
#@title MCMC
num_results = 500
num_burnin_steps = 300

start = joint.sample()

# Improve performance by tracing the sampler using `tf.function`
# and compiling it using XLA.
@tf.function(autograph=False, experimental_compile=True)
def do_sampling():

  # kernel = tfp.mcmc.TransformedTransitionKernel(inner_kernel=tfp.mcmc.HamiltonianMonteCarlo(
  #         target_log_prob_fn=target_log_prob_fn,
  #         step_size=0.04,
  #         num_leapfrog_steps=3),bijector= [tfb.Identity(), tfb.Sigmoid(low=1.0, high=5.0), tfb.Softplus()])

  kernel = tfp.mcmc.NoUTurnSampler(
      target_log_prob_fn=target_log_prob_fn,
      step_size=0.2)


  kernel = tfp.mcmc.TransformedTransitionKernel(inner_kernel=kernel ,
      bijector= [tfb.Identity(), tfb.Sigmoid(low=1.0, high=5.0), tfb.Softplus()])
  kernel = tfp.mcmc.DualAveragingStepSizeAdaptation(
      inner_kernel=kernel,
      target_accept_prob=0.85,
      num_adaptation_steps=tf.cast(0.8*num_burnin_steps, tf.int32),
      step_size_setter_fn=lambda pkr, new_step_size: pkr._replace(inner_results=pkr.inner_results._replace(step_size=new_step_size)),
      step_size_getter_fn=lambda pkr: pkr.inner_results.step_size,
      log_accept_prob_getter_fn=lambda pkr: pkr.inner_results.log_accept_ratio
  )

  return tfp.mcmc.sample_chain(
      num_results=num_results,
      num_burnin_steps=num_burnin_steps,
      current_state=[
          tf.zeros([1,24], name='init_delta'),
          2.0*tf.ones([160,1], name='init_psi'),
          0.1*tf.ones([1,24], name='init_upsilond'),
      ],
      kernel = kernel,
      )

states, kernel_results = do_sampling()

delta, psi, upsilon = states

#num_accepted = np.sum(kernel_results.is_accepted)
#print('Acceptance rate: {}'.format(num_accepted / num_results))

In [ ]:
num_accepted = np.sum(kernel_results.inner_results.inner_results.is_accepted)

#print('Acceptance rate: {}'.format(num_accepted / num_results))
lar = kernel_results.inner_results.inner_results.log_accept_ratio
tf.math.exp(tfp.math.reduce_logmeanexp(lar)).numpy()

In [ ]:
#@title Krytyka
psi_hat = np.mean(psi, axis=0)
psi_err = [psi_hat - np.percentile(psi, q=2.5, axis=0),
      np.percentile(psi, q=97.5, axis=0) - psi_hat]
psi_err[0] = psi_err[0].transpose()
psi_err[1] = psi_err[1].transpose()
psi_err = [psi_err[0][0], psi_err[1][0]]

delta_hat = np.mean(delta, axis=0)
delta_err = [delta_hat - np.percentile(delta, q=2.5, axis=0),
      np.percentile(delta, q=97.5, axis=0) - delta_hat]
delta_err = [delta_err[0][0], delta_err[1][0]]

upsilon_hat = np.mean(upsilon, axis=0)
upsilon_err = [upsilon_hat - np.percentile(upsilon, q=2.5, axis=0),
      np.percentile(upsilon, q=97.5, axis=0) - upsilon_hat]
upsilon_err = [upsilon_err[0][0], upsilon_err[1][0]]

make_plot(x[1].numpy().transpose()[0], psi_hat.transpose()[0],
          x[0].numpy()[0], delta_hat[0],
          x[2].numpy()[0], upsilon_hat[0],
          psi_err, delta_err, upsilon_err )

# Ruch

Ruch generowany z AGH do Cyfronetu

In [ ]:
import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt


from tensorflow_probability import distributions as tfd
from tensorflow_probability import sts
import pandas as pd

In [ ]:
#@title Ruch UCI- Cyfronet (agregacja  2 h)


json2h = '{"ucirtr-cyfronet":{"1568419200000":13.8812913895,"1568484000000":11.0352973938,"1568491200000":6.2486467361,"1568498400000":4.4284286499,"1568505600000":7.4588785172,"1568512800000":10.1032447815,"1568520000000":10.2605657578,"1568527200000":9.5760698318,"1568534400000":12.0147314072,"1568541600000":11.5504827499,"1568548800000":12.5885190964,"1568556000000":10.086602211,"1568563200000":12.9296092987,"1568570400000":8.7450046539,"1568577600000":7.6230592728,"1568584800000":7.1903991699,"1568592000000":4.8369817734,"1568599200000":13.0057868958,"1568606400000":11.1664953232,"1568613600000":9.6513280869,"1568620800000":20.32289505,"1568628000000":26.4453821182,"1568635200000":20.6952829361,"1568642400000":17.9827976227,"1568649600000":12.7699565887,"1568656800000":10.9109830856,"1568664000000":9.0148973465,"1568671200000":6.9933071136,"1568678400000":5.1951265335,"1568685600000":12.1466197968,"1568692800000":15.6340675354,"1568700000000":11.9963150024,"1568707200000":26.0833673477,"1568714400000":23.9553031921,"1568721600000":29.1982059479,"1568728800000":25.1304140091,"1568736000000":21.7263269424,"1568743200000":33.5963592529,"1568750400000":26.951751709,"1568757600000":20.3248443604,"1568764800000":23.5137281418,"1568772000000":21.4717874527,"1568779200000":23.9479751587,"1568786400000":17.9985208511,"1568793600000":27.0015211105,"1568800800000":22.8699426651,"1568808000000":32.4719715118,"1568815200000":33.9049739838,"1568822400000":25.1363830566,"1568829600000":20.2521429062,"1568836800000":10.8851823807,"1568844000000":8.0663862228,"1568851200000":6.4247989655,"1568858400000":17.9046840668,"1568865600000":15.2988767624,"1568872800000":22.1933517456,"1568880000000":27.2937049866,"1568887200000":20.4232053757,"1568894400000":20.9038877487,"1568901600000":23.1486787796,"1568908800000":14.1391687393,"1568916000000":13.7681703568,"1568923200000":13.326499939,"1568930400000":11.5650157928,"1568937600000":13.039390564,"1568944800000":20.6501483917,"1568952000000":12.4003791809,"1568959200000":9.6910419464,"1568966400000":19.9819669724,"1568973600000":28.3414821625,"1568980800000":25.8480358124,"1568988000000":23.7866411209,"1568995200000":25.6241464615,"1569002400000":14.0703363419,"1569009600000":12.3194084167,"1569016800000":10.1479005814,"1569024000000":14.0126752853,"1569031200000":14.2945652008,"1569038400000":9.9634084702,"1569045600000":10.9000167847,"1569052800000":11.1834888458,"1569060000000":10.5376119614,"1569067200000":12.0187959671,"1569074400000":9.7190580368,"1569081600000":10.8790817261,"1569088800000":12.640414238,"1569096000000":7.9628562927,"1569103200000":5.2612762451,"1569110400000":12.5434770584,"1569117600000":14.2433385849,"1569124800000":10.0728883743,"1569132000000":8.7672948837,"1569139200000":6.6516513824,"1569146400000":11.6201257706,"1569153600000":8.5870542526,"1569160800000":9.3648853302,"1569168000000":12.856253624,"1569175200000":13.0635089874,"1569182400000":13.719707489,"1569189600000":9.9570846558,"1569196800000":6.305062294,"1569204000000":16.6281318665,"1569211200000":13.4098577499,"1569218400000":13.0928649902,"1569225600000":24.6390037537,"1569232800000":26.4503202438,"1569240000000":27.5384273529,"1569247200000":28.2279453278,"1569254400000":23.3886117935,"1569261600000":23.31224823,"1569268800000":15.9882678986,"1569276000000":11.3133640289,"1569283200000":6.6927013397,"1569290400000":18.601931572,"1569297600000":11.8167819977,"1569304800000":9.7702436447,"1569312000000":30.5572223663,"1569319200000":33.3979091644,"1569326400000":29.4945392609,"1569333600000":37.1393260956,"1569340800000":24.0078554153,"1569348000000":21.005616188,"1569355200000":24.3301763535,"1569362400000":18.1522693634,"1569369600000":7.2114696503,"1569376800000":19.9837636948,"1569384000000":11.905175209,"1569391200000":11.7033729553,"1569398400000":28.9921731949,"1569405600000":37.3940515518,"1569412800000":23.7961025238,"1569420000000":21.4920444489,"1569427200000":15.2702541351,"1569434400000":27.2681322098,"1569441600000":21.9165973663,"1569448800000":14.2580490112,"1569456000000":5.7384471893,"1569463200000":2.8377552032,"1569470400000":5.3327140808,"1569477600000":5.9125795364,"1569484800000":16.5542516708,"1569492000000":19.0557470322,"1569499200000":22.0035457611,"1569506400000":25.3532152176,"1569513600000":17.337015152,"1569520800000":18.6572170258,"1569528000000":15.6224088669,"1569535200000":9.6441555023,"1569542400000":7.5662403107,"1569549600000":2.6372098923,"1569556800000":5.171880722,"1569564000000":5.7891178131,"1569571200000":15.4593725204,"1569578400000":21.5705881119,"1569585600000":18.5547809601,"1569592800000":16.9213562012,"1569600000000":12.9143133163,"1569607200000":14.4320344925,"1569614400000":8.8227396011,"1569621600000":10.7910251617,"1569628800000":6.5996084213,"1569636000000":2.8027820587,"1569643200000":7.6807746887,"1569650400000":3.2323360443,"1569657600000":4.874753952,"1569664800000":8.0501356125,"1569672000000":9.5251960754,"1569679200000":7.5813055038,"1569686400000":8.2039604187,"1569693600000":7.3430643082,"1569700800000":12.1938705444,"1569708000000":8.4164361954,"1569715200000":6.8458909988,"1569722400000":3.2366456985,"1569729600000":5.1908473969,"1569736800000":2.9616680145,"1569744000000":6.5994129181,"1569751200000":7.633477211,"1569758400000":7.6894264221,"1569765600000":7.6344594955,"1569772800000":9.2367572784,"1569780000000":12.208404541,"1569787200000":12.1237697601,"1569794400000":9.5196247101,"1569801600000":9.3004493713,"1569808800000":2.7369775772,"1569816000000":5.1401596069,"1569823200000":10.365606308,"1569830400000":22.0133628845,"1569837600000":24.9719324112,"1569844800000":20.7215280533,"1569852000000":25.1635456085,"1569859200000":16.0612545013,"1569866400000":11.2539157867,"1569873600000":14.2481622696,"1569880800000":10.025809288,"1569888000000":6.8187999725,"1569895200000":3.6083250046,"1569902400000":6.2369012833,"1569909600000":7.1601362228,"1569916800000":15.9780950546,"1569924000000":23.9845056534,"1569931200000":22.1382341385,"1569938400000":19.1896095276,"1569945600000":24.7612123489,"1569952800000":12.803103447,"1569960000000":14.1146335602,"1569967200000":12.8642911911,"1569974400000":8.2895212173,"1569981600000":2.6373748779,"1569988800000":5.3775348663,"1569996000000":7.7182703018,"1570003200000":23.6170682907,"1570010400000":29.316113472,"1570017600000":22.1752500534,"1570024800000":19.1364650726,"1570032000000":15.8650541306,"1570039200000":17.507566452,"1570046400000":14.6064043045,"1570053600000":12.2799386978,"1570060800000":7.1964435577,"1570068000000":3.939002037,"1570075200000":6.4886045456,"1570082400000":9.1248750687,"1570089600000":16.9130315781,"1570096800000":19.6200809479,"1570104000000":20.4392604828,"1570111200000":23.8722324371,"1570118400000":20.7750234604,"1570125600000":17.6957588196,"1570132800000":14.1465873718,"1570140000000":8.6564025879,"1570147200000":4.5389070511,"1570154400000":2.4019441605,"1570161600000":5.0965604782,"1570168800000":4.9199953079,"1570176000000":21.6105718613,"1570183200000":23.3055534363,"1570190400000":25.7109384537,"1570197600000":19.9720029831,"1570204800000":13.1611824036,"1570212000000":11.0197067261,"1570219200000":12.6574354172,"1570226400000":13.265045166,"1570233600000":5.1473751068,"1570240800000":2.4796943665,"1570248000000":4.934753418,"1570255200000":3.1080121994,"1570262400000":7.2375364304,"1570269600000":11.0372915268,"1570276800000":11.3385868073,"1570284000000":16.6236352921,"1570291200000":10.1193752289,"1570298400000":14.8094654083,"1570305600000":11.7030076981,"1570312800000":10.8857269287,"1570320000000":7.7032728195,"1570327200000":4.1217699051,"1570334400000":5.5705537796,"1570341600000":3.2560358047,"1570348800000":7.6042842865,"1570356000000":11.7265586853,"1570363200000":10.1741657257,"1570370400000":10.0460863113,"1570377600000":13.0388679504,"1570384800000":12.4281740189,"1570392000000":8.6007261276,"1570399200000":8.993970871,"1570406400000":4.7833538055,"1570413600000":2.2041845322,"1570420800000":4.4672689438,"1570428000000":6.9021177292,"1570435200000":20.2461042404,"1570442400000":22.8364505768,"1570449600000":24.7153711319,"1570456800000":15.6544179916,"1570464000000":11.9599552155,"1570471200000":14.679104805,"1570478400000":14.0240697861,"1570485600000":9.7946748734,"1570492800000":5.6736497879,"1570500000000":2.3783426285,"1570507200000":5.3447647095,"1570514400000":5.7032279968,"1570521600000":20.5710763931,"1570528800000":21.802488327,"1570536000000":21.5783090591,"1570543200000":15.9764490128,"1570550400000":21.3822927475,"1570557600000":15.3416852951,"1570564800000":14.5169963837,"1570572000000":12.5067119598,"1570579200000":6.916683197,"1570586400000":4.1674985886,"1570593600000":7.1959066391,"1570600800000":7.8605957031,"1570608000000":19.7534303665,"1570615200000":24.505188942,"1570622400000":24.3192005157,"1570629600000":16.5487623215,"1570636800000":11.8100147247,"1570644000000":13.9470300674,"1570651200000":15.7521047592,"1570658400000":12.1316661835,"1570665600000":9.089471817,"1570672800000":5.392827034,"1570680000000":6.628159523,"1570687200000":8.1984491348,"1570694400000":17.5958375931,"1570701600000":20.179145813,"1570708800000":26.3509111404,"1570716000000":15.4401578903,"1570723200000":12.6684808731,"1570730400000":18.3755521774,"1570737600000":11.778793335,"1570744800000":13.1060886383,"1570752000000":7.1794948578,"1570759200000":2.9544706345,"1570766400000":5.3222665787,"1570773600000":5.5329732895,"1570780800000":17.2517766953,"1570788000000":17.3663578033,"1570795200000":16.456612587,"1570802400000":16.6024236679,"1570809600000":9.2942523956,"1570816800000":15.7696275711,"1570824000000":8.0588531494,"1570831200000":9.6728048325,"1570838400000":6.6648254395,"1570845600000":2.6121330261,"1570852800000":5.0993080139,"1570860000000":3.4924983978,"1570867200000":9.7563753128,"1570874400000":10.6898012161,"1570881600000":12.7292938232,"1570888800000":14.7486782074,"1570896000000":13.8794698715,"1570903200000":11.577249527,"1570910400000":14.7140455246,"1570917600000":11.1145009995,"1570924800000":5.5397319794,"1570932000000":3.4924793243,"1570939200000":5.4766302109,"1570946400000":3.5265808105,"1570953600000":7.2327260971,"1570960800000":10.4751739502,"1570968000000":13.5038318634,"1570975200000":9.4010753632,"1570982400000":8.2029914856,"1570989600000":9.5338945389,"1570996800000":11.3425827026,"1571004000000":8.9408321381,"1571011200000":5.482257843,"1571018400000":2.9534053802,"1571025600000":5.5936641693,"1571032800000":5.2609395981,"1571040000000":18.3318853378,"1571047200000":24.7134208679,"1571054400000":20.5989017487,"1571061600000":17.5909719467,"1571068800000":19.4495515823,"1571076000000":21.823633194,"1571083200000":17.4881505966,"1571090400000":9.3081893921,"1571097600000":5.3554830551,"1571104800000":2.9083824158,"1571112000000":6.0738668442,"1571119200000":9.1592855453,"1571126400000":20.3182621002,"1571133600000":25.5191869736,"1571140800000":20.8043699265,"1571148000000":20.5155391693,"1571155200000":14.9231891632,"1571162400000":12.8500213623,"1571169600000":14.8346776962,"1571176800000":8.3158340454,"1571184000000":11.0519733429,"1571191200000":2.9530649185,"1571198400000":5.4310846329,"1571205600000":8.6660337448,"1571212800000":20.6873235703,"1571220000000":19.5721187592,"1571227200000":29.5884799957,"1571234400000":18.834728241,"1571241600000":16.9456720352,"1571248800000":14.2864789963,"1571256000000":13.6990814209,"1571263200000":11.88194561,"1571270400000":5.3476896286,"1571277600000":2.4880552292,"1571284800000":5.1247386932,"1571292000000":7.4525518417,"1571299200000":20.8080310822,"1571306400000":23.9048318863,"1571313600000":18.1881561279,"1571320800000":22.2651386261,"1571328000000":12.0372829437,"1571335200000":18.149523735,"1571342400000":19.3957939148,"1571349600000":11.9152612686,"1571356800000":8.7257509232,"1571364000000":2.1863851547,"1571371200000":5.3774471283,"1571378400000":6.1402873993,"1571385600000":17.4604845047,"1571392800000":21.3077630997,"1571400000000":22.1562767029,"1571407200000":16.2325029373,"1571414400000":11.2044630051,"1571421600000":6.8642444611,"1571428800000":12.4044218063,"1571436000000":7.3247184753,"1571443200000":4.709277153,"1571450400000":2.1518230438,"1571457600000":4.7358026505,"1571464800000":3.1198444366,"1571472000000":4.3567075729,"1571479200000":5.5234985352,"1571486400000":8.7403278351,"1571493600000":9.7795419693,"1571500800000":9.6932840347,"1571508000000":9.557890892,"1571515200000":18.575884819,"1571522400000":9.1176052094,"1571529600000":5.0487909317,"1571536800000":3.5350131989,"1571544000000":4.7292490005,"1571551200000":2.7401590347,"1571558400000":5.3502025604,"1571565600000":7.2052898407,"1571572800000":8.816942215,"1571580000000":10.0457305908,"1571587200000":8.556227684,"1571594400000":10.2668924332,"1571601600000":12.3172540665,"1571608800000":9.0496177673,"1571616000000":7.2980775833,"1571623200000":4.8488092422,"1571630400000":4.6348867416,"1571637600000":7.7126379013,"1571644800000":23.9126873016,"1571652000000":32.7485599518,"1571659200000":31.3188142776,"1571666400000":22.9081277847,"1571673600000":21.1761302948,"1571680800000":19.1982707977,"1571688000000":15.8477783203,"1571695200000":12.832034111,"1571702400000":9.2877416611,"1571709600000":5.7299642563,"1571716800000":8.2294616699,"1571724000000":15.0933675766,"1571731200000":25.2384405136,"1571738400000":32.9571037292,"1571745600000":23.8820667267,"1571752800000":20.0696325302,"1571760000000":17.3383150101,"1571767200000":18.8409166336,"1571774400000":15.7347183228,"1571781600000":9.9683828354,"1571788800000":8.9345369339,"1571796000000":2.9880771637,"1571803200000":6.7207832336,"1571810400000":6.9090528488,"1571817600000":23.7218341827,"1571824800000":24.1959409714,"1571832000000":21.7843961716,"1571839200000":20.4458417892,"1571846400000":15.9523200989,"1571853600000":19.0020513535,"1571860800000":21.0707960129,"1571868000000":14.0452775955,"1571875200000":7.9066543579,"1571882400000":6.2004785538,"1571889600000":6.7374038696,"1571896800000":13.8077373505,"1571904000000":20.6499729156,"1571911200000":29.8491783142,"1571918400000":23.9180021286,"1571925600000":19.2500658035,"1571932800000":13.1246213913,"1571940000000":14.1152582169,"1571947200000":19.3348855972,"1571954400000":12.5854606628,"1571961600000":10.5439291,"1571968800000":2.8763790131,"1571976000000":4.6800336838,"1571983200000":7.5358247757,"1571990400000":20.2793188095,"1571997600000":29.073718071,"1572004800000":18.166387558,"1572012000000":20.8513507843,"1572019200000":11.3443603516,"1572026400000":12.6847352982,"1572033600000":12.7086591721,"1572040800000":10.312169075,"1572048000000":6.7304544449,"1572055200000":2.7490177155,"1572062400000":2.2476530075,"1572069600000":3.243897438,"1572076800000":6.6803712845,"1572084000000":13.5723571777,"1572091200000":13.5437498093,"1572098400000":21.4667320251,"1572105600000":13.058681488,"1572112800000":14.1381206512,"1572120000000":12.2642946243,"1572127200000":13.5715293884,"1572134400000":8.4687852859,"1572141600000":2.8592624664,"1572148800000":1.8619232178,"1572156000000":2.8370218277,"1572163200000":4.8008165359,"1572170400000":10.1619510651,"1572177600000":11.3489580154,"1572184800000":9.8906574249,"1572192000000":10.6303052902,"1572199200000":9.8005437851,"1572206400000":14.936290741,"1572213600000":9.4674158096,"1572220800000":4.9061946869,"1572228000000":2.0422954559,"1572235200000":1.8867855072,"1572242400000":5.1267843246,"1572249600000":24.6007347107,"1572256800000":23.3287525177,"1572264000000":21.3160142899,"1572271200000":28.5190916061,"1572278400000":23.9989719391,"1572285600000":16.5455007553,"1572292800000":17.7693901062,"1572300000000":17.4354543686,"1572307200000":9.955452919,"1572314400000":2.9475078583,"1572321600000":5.7065629959,"1572328800000":10.9637746811,"1572336000000":23.6299753189,"1572343200000":25.8366756439,"1572350400000":34.5739488602,"1572357600000":29.3032197952,"1572364800000":19.1806745529,"1572372000000":18.0369911194,"1572379200000":12.885515213,"1572386400000":16.5995855331,"1572393600000":10.7528295517,"1572400800000":4.3026809692,"1572408000000":4.9075717926,"1572415200000":13.3496026993,"1572422400000":31.2643318176,"1572429600000":27.2750844955,"1572436800000":22.9317188263,"1572444000000":15.4402751923,"1572451200000":23.8945083618,"1572458400000":13.198592186,"1572465600000":12.9878997803,"1572472800000":14.4293737411,"1572480000000":11.3081712723,"1572487200000":9.4123764038,"1572494400000":3.2451953888,"1572501600000":6.9505434036,"1572508800000":15.7283859253,"1572516000000":22.8620824814,"1572523200000":30.130150795,"1572530400000":17.2585096359,"1572537600000":14.9199886322,"1572544800000":11.7326660156,"1572552000000":10.7741146088,"1572559200000":15.59126091,"1572566400000":13.5456905365,"1572573600000":8.175822258,"1572580800000":4.8282575607,"1572588000000":6.9497251511,"1572595200000":5.1133012772,"1572602400000":7.3733253479,"1572609600000":7.1313056946,"1572616800000":11.3271722794,"1572624000000":9.0641727448,"1572631200000":9.4207458496,"1572638400000":9.4373292923,"1572645600000":11.5541305542,"1572652800000":6.2993421555,"1572660000000":4.673956871,"1572667200000":5.395067215,"1572674400000":5.197766304,"1572681600000":2.8141040802,"1572688800000":4.9831514359,"1572696000000":17.6886529922,"1572703200000":10.0749740601,"1572710400000":7.3815755844,"1572717600000":9.6316261292,"1572724800000":10.6639690399,"1572732000000":9.4382743835,"1572739200000":10.2301435471,"1572746400000":3.2403116226,"1572753600000":2.4927845001,"1572760800000":4.5240907669,"1572768000000":2.9409208298,"1572775200000":7.4866085052,"1572782400000":8.6903114319,"1572789600000":14.555305481,"1572796800000":10.3293714523}}'

In [ ]:
uci = pd.read_json(json2h)
uci.plot()

## Linear state space model

In [ ]:
ruci = uci['ucirtr-cyfronet'].to_numpy() #[Mb/s]
dates = uci.index.to_numpy()
num_forecast_steps=7*12 # tydzien
training_data = ruci[:-num_forecast_steps]


In [ ]:
#@title Dane do nauki
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(1, 1, 1)
ax.plot(
    dates,
    ruci, lw=2, label="training data")
plt.legend();

In [ ]:
def build_model(observed_time_series):
  trend = sts.LocalLinearTrend(observed_time_series=observed_time_series)
  seasonal_weakly = tfp.sts.Seasonal(
      num_seasons=7,
      num_steps_per_season=12,
      observed_time_series=observed_time_series, name="weakly")

  seasonal_dayly = tfp.sts.Seasonal(
    num_seasons=12,
    observed_time_series=observed_time_series,
    name="dayly")
  # seasonal_dayly = tfp.sts.SmoothSeasonal(period=12,
  #                                         frequency_multipliers=[1,2,3], observed_time_series=observed_time_series,,
  #                                         name="dayly")
  autoregressive = sts.Autoregressive(
    order=1,
    observed_time_series=observed_time_series,
    name='autoregressive')

  model = sts.Sum([autoregressive,
                   seasonal_weakly,
                   seasonal_dayly,
                   trend],
                  observed_time_series=observed_time_series)
  return model

In [ ]:
model = build_model(training_data)

variational_posteriors = tfp.sts.build_factored_surrogate_posterior(
    model=model)

In [ ]:
num_variational_steps = 200 # @param { isTemplate: true}
num_variational_steps = int(num_variational_steps)

optimizer = tf.optimizers.Adam(learning_rate=.1)

@tf.function(experimental_compile=True)
def train():
    elbo_loss_curve = tfp.vi.fit_surrogate_posterior(
    target_log_prob_fn=model.joint_log_prob(
        observed_time_series=training_data),
    surrogate_posterior=variational_posteriors,
    optimizer=optimizer,
    num_steps=num_variational_steps)
    return elbo_loss_curve

elbo_loss_curve = train()

plt.plot(elbo_loss_curve)
plt.show()

# Draw samples from the variational posterior.
q_samples_ = variational_posteriors.sample(50)

In [ ]:
for param in model.parameters:
      print("{}: {} +- {}".format(param.name,
                              np.mean(q_samples_[param.name], axis=0),
                              np.std(q_samples_[param.name], axis=0)))

## Predykcja

In [ ]:
forecast_dist = tfp.sts.forecast(
    model,
    observed_time_series=training_data,
    parameter_samples=q_samples_,
    num_steps_forecast=num_forecast_steps)

In [ ]:
num_samples=100

forecast_mean, forecast_scale, forecast_samples = (
    forecast_dist.mean().numpy()[..., 0],
    forecast_dist.stddev().numpy()[..., 0],
    forecast_dist.sample(num_samples).numpy()[..., 0])

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(1, 1, 1)
ax.plot(dates[-num_forecast_steps:],ruci[-num_forecast_steps:],label="true")
forecast_steps = dates[-num_forecast_steps:]
ax.plot(forecast_steps,forecast_mean,label="Forecast")
#ax.plot(forecast_steps, forecast_samples.T, lw=1, alpha=0.1);
ax.axvline(dates[-num_forecast_steps], linestyle="--")
ax.fill_between(forecast_steps,
                   forecast_mean-2*forecast_scale,
                   forecast_mean+2*forecast_scale, alpha=0.2, label=r"$\pm 2\sigma$")
# ax.plot(
#     dates[:-num_forecast_steps],
#     ruci[:-num_forecast_steps], lw=2, label="training data")

ax.legend()

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(1, 1, 1)
ax.plot(dates[-num_forecast_steps:],ruci[-num_forecast_steps:],label="true")
forecast_steps = dates[-num_forecast_steps:]
ax.plot(forecast_steps,forecast_mean,label="Forecast")
#ax.plot(forecast_steps, forecast_samples.T, lw=1, alpha=0.1);
ax.axvline(dates[-num_forecast_steps], linestyle="--")
ax.fill_between(forecast_steps,
                   forecast_mean-2*forecast_scale,
                   forecast_mean+2*forecast_scale, alpha=0.2, label=r"$\pm 2\sigma$")
ax.plot(
    dates[:-num_forecast_steps],
    ruci[:-num_forecast_steps], lw=2, label="training data")

ax.legend()

### Składowe

In [ ]:
component_dists = sts.decompose_by_component(
    model,
    observed_time_series=training_data,
    parameter_samples=q_samples_)

In [ ]:
component_means_, component_stddevs_ = (
    {k.name: c.mean() for k, c in component_dists.items()},
    {k.name: c.stddev() for k, c in component_dists.items()})

comps = pd.DataFrame(component_means_, index=uci.index[:-num_forecast_steps])

In [ ]:
comps.plot(subplots=True, figsize=(12, 8));

In [ ]:
comps.head(5*12).plot(subplots=True, figsize=(12, 8));

In [ ]:
comps.tail(5*12).plot(subplots=True, figsize=(12, 8));